In [1]:
import copy
import itertools
import numpy as np
import glob
import h5py
import scipy
import sys
import tqdm
import verdict
import os
import unyt

In [2]:
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import palettable

In [3]:
import kalepy as kale

In [4]:
from py2tex import py2tex

In [5]:
import galaxy_dive.analyze_data.halo_data as halo_data
import galaxy_dive.plot_data.plotting as plotting

In [6]:
import trove
import trove.config_parser

In [7]:
import helpers

# Load Data

In [8]:
pm = {
    'spin_parameter': 0.035,
    'used_variations': [
        'm12i',
        'm12m',
        'm11a',
        'm11b',
        'm11c',
        'm12i_md',
        'm12r_md',
        'm12w_md',
        'm12f_md',
        'm12z_md',
        'm12c_md',
        'm12b_md',
        'm11d_md',
        'm11e_md',
        'm11h_md',
        'm11i_md',
        'm11q_md',
        # 'm12i_cr',
    ],
    'include_proportional_line': False,
    'snum': 600,
}

In [9]:
config_fp = '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove'
cp = trove.config_parser.ConfigParser( config_fp )
pm = trove.link_params_to_config(
    config_fp,
    variation = 'm12i_md',
    global_variation = '',
    **pm
)

In [10]:
base_processed_data_dir = pm['config_parser'].get( 'DEFAULT', 'processed_data_dir' )
default_data_fp = os.path.join( base_processed_data_dir, 'summary.hdf5' )
default_data = verdict.Dict.from_hdf5( default_data_fp, create_nonexistent=True )

/home1/03057/zhafen/repos/verdict/verdict.py:1407: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if hasattr( a[first_element_index][0], 'decode' ):


In [11]:
data_filepath = os.path.join( pm['processed_data_dir'], 'summary.hdf5' )
data = verdict.Dict.from_hdf5( data_filepath )

In [12]:
data['before_after_plot_data'].keys()

dict_keys(['m11a', 'm11b', 'm11c', 'm11d_md', 'm11e_md', 'm11h_md', 'm11i_md', 'm11q_md', 'm12b_md', 'm12c_md', 'm12f_md', 'm12i', 'm12i_cr', 'm12i_md', 'm12m', 'm12r_md', 'm12w_md', 'm12z_md'])

In [13]:
h_param = .702

# Extract Quantities

## Halo Quantities

In [14]:
mvir = verdict.Dict({})
mstar = verdict.Dict({})
vc = verdict.Dict({})
rstar = verdict.Dict({})
rvir = verdict.Dict({})
for key in tqdm.tqdm( pm['used_variations'] ):
    
    # Load from summary data if available
    all_data_types_in_summary_data = True
    for data_type in [ 'Mvir', 'Mstar', 'Vc', 'Rstar0.5', 'Rvir' ]:
        if data_type not in data:
            data[data_type] = {}
        all_data_types_in_summary_data = all_data_types_in_summary_data & ( key in data[data_type] )
    if all_data_types_in_summary_data:
        mvir[key] = data['Mvir'][key]
        mstar[key] = data['Mstar'][key]
        vc[key] = data['Vc'][key]
        rstar[key] = data['Rstar0.5'][key]
        rvir[key] = data['Rvir'][key]
        continue
    
    pm_i = trove.link_params_to_config(
        '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
        variation = key,
        global_variation = pm['global_variation'],
    )
    
    h_data = halo_data.HaloData(
        data_dir = pm_i['halo_data_dir'],
        mt_kwargs = { 'tag': 'smooth' },
    )
    
    mvir[key] =  h_data.get_mt_data( 'Mvir', snums=[600,] )[0] / h_param
    mstar[key] = h_data.get_mt_data( 'M_star', snums=[600,] )[0] / h_param
    vc[key] = h_data.get_mt_data( 'Vmax', snums=[600,] )[0]
    rstar[key] = h_data.get_mt_data( 'Rstar0.5', snums=[600,] )[0] / h_param
    rvir[key] = h_data.get_mt_data( 'Rvir', snums=[600,] )[0] / h_param
    
    data['Mvir'][key] = mvir[key]
    data['Mstar'][key] = mstar[key]
    data['Vc'][key] = vc[key]
    data['Rstar0.5'][key] = rstar[key]
    data['Rvir'][key] = rvir[key]
    
# if not all_data_types_in_summary_data:
#     data.to_hdf5( data_filepath, handle_jagged_arrs='row datasets' )

100%|██████████| 17/17 [00:05<00:00,  3.33it/s]


In [15]:
jdm = pm['spin_parameter'] * verdict.Dict( data['Vc'] ) * verdict.Dict( data['Rvir'] ) * np.sqrt(2.)

## tcool / tff

In [ ]:
# tcool/tff
missing = []
tcool_tff = verdict.Dict({})
for key in pm['used_variations']:
    
    try:
        # Get the file
        res = cp.get( key, 'subpath' ).split( '_' )[-1][3:]
        fp = os.path.join( base_processed_data_dir, 'tcool_tff', 't_cool_to_t_ff_{}_{}.npz'.format( key, res ) )
        tcool_tff_file = np.load( fp )
    except FileNotFoundError:
        missing.append( key )
        
        continue
        
    tcool_tff[key] = 10.**tcool_tff_file['log_t_cool_to_t_ff_smooth'][0]

## Vrot / sigma

In [ ]:
# tcool/tff
missing = []
vrot_sigma = verdict.Dict({})
for key in pm['used_variations']:
    
    try:
        # Get the file
        res = cp.get( key, 'subpath' ).split( '_' )[-1][3:]
        fp = os.path.join( pm['processed_data_dir'], 'Vrot_to_sigma', 'Vrot_to_sigma_{}_{}.npz'.format( key, res ) )
        vrot_sigma_file = np.load( fp )
    except FileNotFoundError:
        missing.append( key )
        
        continue
        
    vrot_sigma[key] = vrot_sigma_file['Vrot_to_sigmas'][0]

## Disk Fractions

In [ ]:
# Width change
sigma_cosphi = data['cosphi']['84th_percentile'] - data['cosphi']['16th_percentile']
delta_sigma_cosphi = sigma_cosphi.inner_item( -1 ) - sigma_cosphi.inner_item( 0 )
negative_delta_sigma_cosphi = -1. * delta_sigma_cosphi
sigma_cosphi_ratio = sigma_cosphi.inner_item( 0 ) / sigma_cosphi.inner_item( -1 )
std_cosphi_ratio = data['cosphi']['std'].inner_item( 0 ) / data['cosphi']['std'].inner_item( -1 )

In [ ]:
# PDF value changes
delta_pdf = data['cosphi']['pdf(cos theta=0)'].inner_item( -1 ) - data['cosphi']['pdf(cos theta=0)'].inner_item( 0 )
ratio_pdf = data['cosphi']['pdf(cos theta=0)'].inner_item( -1 ) / data['cosphi']['pdf(cos theta=0)'].inner_item( 0 )

In [ ]:
# Spherical harmonics
delta_q20 = data['cosphi']['q20'].inner_item( -1 ) - data['cosphi']['q20'].inner_item( 0 )
delta_q33 = data['cosphi']['q33'].inner_item( -1 ) - data['cosphi']['q33'].inner_item( 0 )

In [ ]:
# Disk fraction
disk_frac = verdict.Dict({})
smooth_disk_frac = verdict.Dict({})
delta_disk_frac = verdict.Dict({})
delta_smooth_disk_frac = verdict.Dict({})
for key in pm['used_variations']:
    disk_fracs = []
    for i, pdf in enumerate( data['cosphi']['pdf'][key] ):
        
        in_disk = np.abs( data['cosphi']['points'][key] ) < pm['disk_costheta']
        disk_fracs.append( pdf[in_disk].sum()/pdf.sum() )
    disk_frac[key] = disk_fracs
    
    smooth_disk_frac[key] = scipy.signal.savgol_filter(
        disk_fracs,
        window_length = 9,
        polyorder = 1,
    )
    
    interp = scipy.interpolate.interp1d( data['cosphi']['t_tchange_centers'][key] * 1e3, disk_frac[key] )
    delta_disk_frac[key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )
    interp = scipy.interpolate.interp1d( data['cosphi']['t_tchange_centers'][key] * 1e3, smooth_disk_frac[key] )
    delta_smooth_disk_frac[key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )

In [ ]:
# Angular Momentum Disk fraction
jdisk_frac = verdict.Dict({})
smooth_jdisk_frac = verdict.Dict({})
delta_jdisk_frac = verdict.Dict({})
delta_smooth_jdisk_frac = verdict.Dict({})
for key in pm['used_variations']:
    jdisk_fracs = []
    for i, pdf in enumerate( data['jzjmag']['pdf'][key] ):
        
        in_disk = data['jzjmag']['points'][key] > pm['disk_jzjmag']
        jdisk_fracs.append( pdf[in_disk].sum()/pdf.sum() )
    jdisk_frac[key] = jdisk_fracs
    
    smooth_jdisk_frac[key] = scipy.signal.savgol_filter(
        jdisk_fracs,
        window_length = 9,
        polyorder = 1,
    )
    
    interp = scipy.interpolate.interp1d( data['jzjmag']['t_tchange_centers'][key] * 1e3, jdisk_frac[key] )
    delta_jdisk_frac[key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )
    interp = scipy.interpolate.interp1d( data['jzjmag']['t_tchange_centers'][key] * 1e3, smooth_jdisk_frac[key] )
    delta_smooth_jdisk_frac[key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )

In [ ]:
# Minimum Disk Fraction
min_delta_disk_frac = verdict.Dict({})
for key, item in delta_disk_frac.items():
    min_delta_disk_frac[key] = min( item, delta_jdisk_frac[key] )

In [ ]:
# Disk fraction (aligned fraction) for thin disk stars, for use as a reference point
aligned_frac = verdict.Dict({})
aligned_frac_recent = verdict.Dict({})
thin_disk_aligned_missing = []
for key in pm['used_variations']:
    aligned = np.abs( default_data['cosphi_stars']['centers'] ) < pm['disk_costheta']
    
    try:
        pdf = default_data['cosphi_stars']['thin_disk'][key]
        aligned_frac[key] = pdf[aligned].sum()/pdf.sum()

        pdf = default_data['cosphi_stars']['thin_disk_recent'][key]
        aligned_frac_recent[key] = pdf[aligned].sum()/pdf.sum()
    except KeyError:
        thin_disk_aligned_missing.append( key )

In [ ]:
# Disk fraction as the fraction under the PDF for recent thin disk stars

# Calculate average pdf
avg_cosphi_stars = (
    ( default_data['cosphi_stars']['thin_disk_recent'] * default_data[pm['thin_disk_frac_key']] ).array().sum( axis=0 ) / 
    default_data[pm['thin_disk_frac_key']].array().sum()
)
# Enforce normalization
avg_cosphi_stars /= avg_cosphi_stars.sum() * ( default_data['cosphi_stars']['centers'][1] - default_data['cosphi_stars']['centers'][0] ) 

disk_frac_pdf = verdict.Dict({})
for key in pm['used_variations']:
    
    # Interpolated to deal with different binning
    avg_cosphi_stars_key = scipy.interpolate.interp1d( default_data['cosphi_stars']['centers'], avg_cosphi_stars )( data['cosphi']['points'][key] )
    
    disk_fracs = []
    for i, pdf in enumerate( data['cosphi']['pdf'][key] ):
        
        # The line underneath
        min_of_both = np.min( [ pdf, avg_cosphi_stars_key], axis=0 )
        
        # The fraction. Equivalent to summing the area
        # (alt code: min_of_both.sum() * data['cosphi']['points'][key][1] - data['cosphi']['points'][key][0] )
        frac = min_of_both.sum() / avg_cosphi_stars_key.sum()

        disk_fracs.append( frac )
        
    disk_frac_pdf[key] = disk_fracs
delta_disk_frac_pdf = disk_frac_pdf.inner_item( -1 ) - disk_frac_pdf.inner_item( 0 )

# Ideal case for scaling
min_of_both_flat = np.min( [ np.full( avg_cosphi_stars.shape,  0.5 ), avg_cosphi_stars], axis=0 )
disk_frac_flat = min_of_both_flat.sum() / avg_cosphi_stars.sum()
delta_disk_frac_ideal = 1. - disk_frac_flat

delta_disk_frac_pdf /= delta_disk_frac_ideal

In [ ]:
# Disk fraction as the fraction under the PDF for all thin disk stars

# Calculate average pdf
avg_cosphi_stars = (
    ( default_data['cosphi_stars']['thin_disk'] * default_data[pm['thin_disk_frac_key']] ).array().sum( axis=0 ) / 
    default_data[pm['thin_disk_frac_key']].array().sum()
)
# Enforce normalization
avg_cosphi_stars /= avg_cosphi_stars.sum() * ( default_data['cosphi_stars']['centers'][1] - default_data['cosphi_stars']['centers'][0] ) 

disk_frac_pdf_all = verdict.Dict({})
for key in pm['used_variations']:
    
    # Interpolated to deal with different binning
    avg_cosphi_stars_key = scipy.interpolate.interp1d( default_data['cosphi_stars']['centers'], avg_cosphi_stars )( data['cosphi']['points'][key] )
    
    disk_fracs = []
    for i, pdf in enumerate( data['cosphi']['pdf'][key] ):
        
        # The line underneath
        min_of_both = np.min( [ pdf, avg_cosphi_stars_key], axis=0 )
        
        # The fraction. Equivalent to summing the area
        # (alt code: min_of_both.sum() * data['cosphi']['points'][key][1] - data['cosphi']['points'][key][0] )
        frac = min_of_both.sum() / avg_cosphi_stars_key.sum()

        disk_fracs.append( frac )
        
    disk_frac_pdf_all[key] = disk_fracs
delta_disk_frac_pdf_all = disk_frac_pdf_all.inner_item( -1 ) - disk_frac_pdf_all.inner_item( 0 )

# Ideal case for scaling
min_of_both_flat = np.min( [ np.full( avg_cosphi_stars.shape,  0.5 ), avg_cosphi_stars], axis=0 )
disk_frac_flat = min_of_both_flat.sum() / avg_cosphi_stars.sum()
delta_disk_frac_ideal_all = 1. - disk_frac_flat

delta_disk_frac_pdf_all /= delta_disk_frac_ideal_all

In [ ]:
# Disk fraction as the fraction under the PDF for all disk stars

# Calculate average pdf
# avg_cosphi_stars = (
#     ( data['cosphi_stars']['thin_disk'] * data[pm['thin_disk_frac_key']] ).array().sum( axis=0 ) / 
#     data[pm['thin_disk_frac_key']].array().sum()
# )
# # Enforce normalization
# avg_cosphi_stars /= avg_cosphi_stars.sum() * ( data['cosphi_stars']['centers'][1] - data['cosphi_stars']['centers'][0] ) 
avg_cosphi_stars = default_data['cosphi_stars']['disk']['m12i_md']

disk_frac_pdf_disk = verdict.Dict({})
for key in pm['used_variations']:
    
    # Interpolated to deal with different binning
    avg_cosphi_stars_key = scipy.interpolate.interp1d( default_data['cosphi_stars']['centers'], avg_cosphi_stars )( data['cosphi']['points'][key] )
    
    disk_fracs = []
    for i, pdf in enumerate( data['cosphi']['pdf'][key] ):
        
        # The line underneath
        min_of_both = np.min( [ pdf, avg_cosphi_stars_key], axis=0 )
        
        # The fraction. Equivalent to summing the area
        # (alt code: min_of_both.sum() * data['cosphi']['points'][key][1] - data['cosphi']['points'][key][0] )
        frac = min_of_both.sum() / avg_cosphi_stars_key.sum()

        disk_fracs.append( frac )
        
    disk_frac_pdf_disk[key] = disk_fracs
delta_disk_frac_pdf_disk = disk_frac_pdf_disk.inner_item( -1 ) - disk_frac_pdf_disk.inner_item( 0 )

# Ideal case for scaling
min_of_both_flat = np.min( [ np.full( avg_cosphi_stars.shape,  0.5 ), avg_cosphi_stars], axis=0 )
disk_frac_flat = min_of_both_flat.sum() / avg_cosphi_stars.sum()
delta_disk_frac_ideal_all = 1. - disk_frac_flat

delta_disk_frac_pdf_disk /= delta_disk_frac_ideal_all

In [ ]:
# Median change
med_cosphi = verdict.Dict({})
abs_med_cosphi = verdict.Dict({})
for key in pm['used_variations']:
    medians = []
    abs_medians = []
    for i, pdf in enumerate( data['cosphi']['pdf'][key] ):

        cdf = np.cumsum( pdf )
        cdf /= cdf[-1]
        medians.append( scipy.interpolate.interp1d( cdf, data['cosphi']['points'][key] )( 0.5 ) )
        
        is_above = data['cosphi']['points'][key] > 0.
        as_above = pdf[is_above]
        so_below = pdf[data['cosphi']['points'][key] < 0.][::-1]
        as_above += so_below
        cdf_abs = np.cumsum( as_above )
        cdf_abs /= cdf_abs[-1]
        abs_medians.append( scipy.interpolate.interp1d( cdf_abs, data['cosphi']['points'][key][is_above] )( 0.5 ) )
        
    med_cosphi[key] = np.array( medians )
    abs_med_cosphi[key] = np.array( abs_medians )

In [ ]:
# Delta of median
delta_med_cosphi = med_cosphi.inner_item( 0 ) - med_cosphi.inner_item( -1 )
delta_abs_med_cosphi = abs_med_cosphi.inner_item( 0 ) - abs_med_cosphi.inner_item( -1 )

## Angular Momentum of Accreting Gas

In [ ]:
dt_pre = pm['aligned_fraction_pre_dt']
dt_post = pm['aligned_fraction_post_dt']
inds2 = np.arange(-50,100)

In [ ]:
delta_jperp = verdict.Dict({})
delta_jz = verdict.Dict({})
delta_jmag = verdict.Dict({})
delta_support = verdict.Dict({})
jmag0 = verdict.Dict({})
for key in pm['used_variations']:
    
    # Retrieve data
    time = data['before_after_plot_data'][key]['time']
    jperp = data['before_after_plot_data'][key]['perpendicular_angular_momentum']['median'][inds2]
    jz = data['before_after_plot_data'][key]['aligned_angular_momentum']['median'][inds2]
    jmag = data['before_after_plot_data'][key]['angular_momentum_magnitude']['median'][inds2]
    support = data['before_after_plot_data'][key]['rotational_support_ratio']['median'][inds2]
    jperp_interp = scipy.interpolate.interp1d( time, jperp )
    jz_interp = scipy.interpolate.interp1d( time, jz )
    jmag_interp = scipy.interpolate.interp1d( time, jmag )
    support_interp = scipy.interpolate.interp1d( time, support )
    
    delta_jperp[key] = jperp_interp( dt_post ) - jperp_interp( dt_pre )
    delta_jz[key] = jz_interp( dt_post ) - jz_interp( dt_pre )
    delta_jmag[key] = jmag_interp( dt_post ) - jmag_interp( dt_pre )
    delta_support[key] = support_interp( dt_post ) - support_interp( dt_pre )
    jmag0[key] = jmag_interp( dt_pre )

In [ ]:
delta_jperp_frac = delta_jperp / jdm

## Torque

In [ ]:
torque_perp = verdict.Dict({})
torque_z = verdict.Dict({})
torque_mag = verdict.Dict({})
for key in pm['used_variations']:

    # Retrieve data
    time = data['before_after_plot_data'][key]['time']
    torqueperp = data['before_after_plot_data'][key]['perpendicular_torque']['median'][inds2]
    torquez = data['before_after_plot_data'][key]['aligned_torque']['median'][inds2]
    torquemag = data['before_after_plot_data'][key]['torque_magnitude']['median'][inds2]
    
    is_in_interval = ( time > dt_pre ) & ( time < dt_post )
    torque_perp[key] = np.nanmean( torqueperp[is_in_interval] )
    torque_z[key] = np.nanmean( torquez[is_in_interval] )
    torque_mag[key] = np.nanmean( torquemag[is_in_interval] )

In [ ]:
torque_normalized = torque_mag / jdm

## Total Angular Momentum

In [ ]:
tot_momentum_fp = os.path.join( base_processed_data_dir, 'tot_momentums.hdf5' )
ang_momentum_data_temp = verdict.Dict.from_hdf5( tot_momentum_fp )
ang_momentum_data = verdict.Dict({})
for key in tqdm.tqdm( pm['used_variations'] ):
    ang_momentum_data[key] = ang_momentum_data_temp[key]
ang_momentum_data = ang_momentum_data.transpose()

In [ ]:
rvc = ( rvir * vc * np.sqrt( 2. ) )
spin_param = ang_momentum_data['Jdmhalo_snum{}'.format( pm['snum'] ) ] / rvc

In [ ]:
f_j_disk = ang_momentum_data['Jthindisk_snum{}'.format( pm['snum'] )] / ang_momentum_data['Jdmhalo_snum{}'.format( pm['snum'] ) ]

In [ ]:
f_j_galaxy = ang_momentum_data['Jstar_snum{}'.format( pm['snum'] )] / ang_momentum_data['Jdmhalo_snum{}'.format( pm['snum'] ) ]

## Finish Setting up Values

In [ ]:
values = {
    'thin_disk_frac': default_data['thin_disk_fraction'],
    'thin_disk_frac_recent': default_data['thin_disk_fraction_recent'],
    'thin_disk_frac_sloanr': default_data['thin_disk_fraction_sloanr'],
    'thin_disk_frac_tracked': default_data['f_thin'],
    'superthin_disk_frac': default_data['superthin_disk_fraction'],
    'superthin_disk_frac_recent': default_data['superthin_disk_fraction_recent'],
    'superthin_disk_frac_sloanr': default_data['superthin_disk_fraction_sloanr'],
    'superthin_disk_frac_tracked': default_data['f_superthin'],
    'thin_disk_frac_recent_orig': default_data['thin_disk_fraction_recent_orig'],
    'thin_disk_frac_recent_ratio': default_data['thin_disk_fraction_recent'] / default_data['thin_disk_fraction_recent_orig'],
    'thin_disk_frac_recent_diff': default_data['thin_disk_fraction_recent'] - default_data['thin_disk_fraction_recent_orig'],
    'superthin_disk_frac_recent_ratio': default_data['superthin_disk_fraction_recent'] / default_data['thin_disk_fraction_recent_orig'],
    'superthin_disk_frac_recent_diff': default_data['superthin_disk_fraction_recent'] - default_data['thin_disk_fraction_recent_orig'],
    'mvir': mvir,
    'mstar': mstar,
    'rstar': rstar,
    'tcool_tff': tcool_tff,
    'negative_delta_sigma_cosphi': negative_delta_sigma_cosphi,
    'sigma_cosphi_ratio': sigma_cosphi_ratio,
    'std_cosphi_ratio': std_cosphi_ratio,
    'delta_pdfcosphi': delta_pdf,
    'ratio_pdfcosphi': ratio_pdf,
    'disk_frac': disk_frac,
    'delta_disk_frac': delta_disk_frac,
    'jdisk_frac': jdisk_frac,
    'delta_jdisk_frac': delta_jdisk_frac,
    'smooth_disk_frac': smooth_disk_frac,
    'delta_smooth_disk_frac': delta_smooth_disk_frac,
    'smooth_jdisk_frac': smooth_jdisk_frac,
    'delta_smooth_jdisk_frac': delta_smooth_jdisk_frac,
    'delta_support': delta_support,
    'min_delta_disk_frac': min_delta_disk_frac,
    'disk_frac_pdf': disk_frac_pdf,
    'delta_disk_frac_pdf': delta_disk_frac_pdf,
    'disk_frac_pdf_all': disk_frac_pdf_all,
    'delta_disk_frac_pdf_all': delta_disk_frac_pdf_all,
    'disk_frac_pdf_disk': disk_frac_pdf_disk,
    'delta_disk_frac_pdf_disk': delta_disk_frac_pdf_disk,
    'delta_q20': delta_q20,
    'delta_q33': delta_q33,
    'delta_med_cosphi': delta_med_cosphi,
    'delta_abs_med_cosphi': delta_abs_med_cosphi,
    'f_hot': default_data['f_hot'],
    'f_valid': default_data['n_valid'] / default_data['n_accreted'],
    'n_hot': default_data['n_hot'],
    'n_tracked': default_data['n_tracked'],
    'n_valid': default_data['n_valid'],
    'delta_jperp': delta_jperp,
    'delta_jperp_frac': delta_jperp_frac,
    'delta_jmag': delta_jmag,
    'delta_jz': delta_jz,
    'torque_perp': torque_perp,
    'torque_z': torque_z,
    'torque_mag': torque_mag,
    'torque_normalized': torque_normalized,
    'spin_param': spin_param,
    'f_j_disk': f_j_disk,
    'f_j_galaxy': f_j_galaxy,
}

# Plot

## Settings

In [ ]:
labels = helpers.quantity_labels

In [ ]:
def delta_f_aligned_label( y_key, label_type='words' ):
    
    f_aligned_label = {
        'delta_smooth_disk_frac': r'f_{\rm aligned}',
        'delta_smooth_jdisk_frac': r'f_{{\rm spin\,aligned}}',
    }[y_key]
    full_tchange_label = helpers.get_tchange_label( pm )
    tchange_label = full_tchange_label[1:-1]
    
    def format_dt( value, use_abs_value=True ):
        
        abs_value = np.abs( value )
        if use_abs_value:
            value = abs_value
        
        if abs_value < 1000:
            formatted_dt = '{} Myr'.format( value )
        else:
            if np.isclose( abs_value, 1000. ):
                formatted_dt = '1 Gyr'
            else:
                formatted_dt = '{} Gyr'.format( value / 1000 )
        return formatted_dt
    
    def sign_text( value ):
        sgn = np.sign( value )
        sign_dict = {
            1: r'+$',
            -1: r'-$',
            0: r'$)',
        }
        sign_text = sign_dict[sgn]
        if sgn != 0:
            sign_text += formatted_dt( value ) + ' )'
        return sign_text
    
    t_post_is_0 = np.isclose(dt_post,0.)
    if t_post_is_0:
        yes_or_no_split_line = ' '
    else:
        yes_or_no_split_line = '\n    '

    if label_type == 'equation':
        text = (
            r'$' + f_aligned_label + r'('
            + tchange_label + sign_text( dt_post )  +
            yes_or_no_split_line + r'$-$ ' +
            r'$' + f_aligned_label + r'('
            + tchange_label + sign_text( dt_pre )
        )
    elif label_type == 'words':
        
        if t_post_is_0:
            range_text = (
                format_dt( dt_pre )
                + ' prior to ' +
                full_tchange_label
            )
        else:
            range_text = (
                format_dt( dt_pre, use_abs_value=False )
                + r' $< t - ' + tchange_label + r'<$' +
                format_dt( dt_post, use_abs_value=False )
            )
        
        text = (
            r'$\Delta ' + f_aligned_label + r'$' +
            ' over\n' + range_text
        )
    
    return text

In [ ]:
logscale = [ 'mvir', 'mstar', 'tcool_tff', 'n_hot', 'n_tracked', 'n_valid', 'f_j_disk', 'f_j_galaxy', 'spin_param' ]

In [ ]:
fractions = [
    'thin_disk_frac',
    'thin_disk_frac_recent',
    'thin_disk_frac_tracked',
    'thin_disk_frac_sloanr',
    'superthin_disk_frac',
    'superthin_disk_frac_recent',
    'superthin_disk_frac_tracked',
    'superthin_disk_frac_sloanr',
    'quiet_frac',
    'quiet_frac_strict',
    'f_hot',
    'f_valid',
    'f_thin',
    'f_superthin',
]

In [ ]:
arrowprops = {
    'arrowstyle': '-',
}

In [ ]:
custom_annot_args = {
    ( 'delta_pdfcosphi', 'thin_disk_frac_recent' ): {
        'm12f_md': {
            'ha': 'right',
            'va': 'bottom',
            'xytext': ( -3, 3 ),
        },
        'm12i_cr': {
            'ha': 'right',
            'va': 'center',
            'xytext': ( -8, 0 ),
        },
        'm12i_md': {
            'ha': 'center',
            'va': 'bottom',
            'xytext': ( 3, 5 ),
        },
        'm12i': {
            'ha': 'left',
            'va': 'top',
            'xytext': ( 3, -3 ),
        },
#         'm11e_md': {
#             'xytext': ( 50, 3 ),
#             'arrowprops': arrowprops,
#         },
#         'm11c': {
#             'xytext': ( 50, 3 ),
#             'arrowprops': arrowprops,
#         },
#         'm11q_md': {
#             'xytext': ( 85, 0 ),
#             'arrowprops': arrowprops,
#             'ha': 'left',
#             'va': 'bottom',
#         },
#         'm11a': {
#             'xytext': ( 0, 65 ),
#             'ha': 'left',
#             'va': 'bottom',
#             'arrowprops': arrowprops,
#         },
#         'm11i_md': {
#             'xytext': ( 0, 40 ),
#             'ha': 'left',
#             'va': 'bottom',
#             'arrowprops': arrowprops,
#         },
#         'm11d_md': {
#             'xytext': ( 20, 25 ),
#             'ha': 'left',
#             'va': 'bottom',
#             'arrowprops': arrowprops,
#         },
        'm11a': None,
        'm11c': None,
        'm11d_md': None,
        'm11e_md': None,
        'm11i_md': None,
        'm11q_md': None,
    },
    ( 'thin_disk_frac_recent', 'delta_disk_frac' ): {
        'm12i': {
            'ha': 'right',
            'va': 'bottom',
            'xytext': ( -3, 3 ),
        },
        'm11a': None,
        'm11c': None,
        'm11d_md': None,
        'm11e_md': None,
        'm11i_md': None,
        'm11q_md': None,
    },
}

In [ ]:
markers = {
    'md': 'o',
#     'cr': '^',
#     'mhdcv': 'P',
    '': 'o',
}

In [ ]:
marker_labels = {
    'md': 'hydro+',
#     'cr': 'CR+',
#     'mhdcv': 'MHD+',
    '': 'no metal diffusion',
}

In [ ]:
custom_lims = helpers.custom_lims

In [ ]:
def get_lim( vs, is_log, v_key, scale_upper=0.1, scale_lower=0.1 ):
    
    if v_key in custom_lims:
        return custom_lims[v_key]
    
    if v_key in fractions:
        return [ 0, 1 ]
        
    max_v = np.nanmax( vs.array() )
    min_v = np.nanmin( vs.array() )
    max_absv = np.nanmax( np.abs([ max_v, min_v]) )
    max_v += max_absv * scale_upper
    min_v -= max_absv * scale_lower
                      
    return min_v, max_v

## One Value per Axis

In [ ]:
# Automatic
value_keys = list( values.keys() )
combinations = itertools.combinations( value_keys, 3 )

In [ ]:
# Manual
combinations = [
#     ( 'thin_disk_frac_recent', 'f_hot', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'n_hot', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'n_tracked', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'n_valid', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'f_valid', 'mvir' ),
    ( 'thin_disk_frac_recent', 'thin_disk_frac_sloanr', 'mvir' ),
    ( 'thin_disk_frac_recent', 'delta_smooth_disk_frac', 'mvir' ),
    ( 'thin_disk_frac_recent', 'delta_smooth_jdisk_frac', 'mvir' ),
    ( 'thin_disk_frac_tracked', 'delta_smooth_disk_frac', 'mvir' ),
    ( 'thin_disk_frac_tracked', 'delta_smooth_jdisk_frac', 'mvir' ),
    ( 'thin_disk_frac_recent', 'thin_disk_frac_tracked', 'mvir' ),
    ( 'superthin_disk_frac_recent', 'superthin_disk_frac_sloanr', 'mvir' ),
    ( 'superthin_disk_frac_recent', 'delta_smooth_disk_frac', 'mvir' ),
    ( 'superthin_disk_frac_recent', 'delta_smooth_jdisk_frac', 'mvir' ),
    ( 'superthin_disk_frac_tracked', 'delta_smooth_disk_frac', 'mvir' ),
    ( 'superthin_disk_frac_tracked', 'delta_smooth_jdisk_frac', 'mvir' ),
    ( 'superthin_disk_frac_recent', 'superthin_disk_frac_tracked', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_jperp', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_jmag', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_jperp_frac', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_perp', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_z', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_mag', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_normalized', 'mvir' ),
]

In [ ]:
# combinations_to_save = [
#     ( 'thin_disk_frac_recent', 'f_hot', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'n_hot', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'n_tracked', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'n_valid', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'thin_disk_frac_sloanr', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_smooth_disk_frac', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_smooth_jdisk_frac', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_jperp', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_jmag', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'delta_jperp_frac', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_perp', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_z', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_mag', 'mvir' ),
#     ( 'thin_disk_frac_recent', 'torque_normalized', 'mvir' ),
# ]
# combinations_no_annotate = combinations_to_save
combinations_to_save = combinations
combinations_no_annotate = combinations

In [ ]:
cmap = palettable.matplotlib.Viridis_20.mpl_colormap

In [ ]:
for i, (x_key, y_key, z_key) in enumerate( tqdm.tqdm( combinations ) ):
    
    xs = values[x_key]
    ys = values[y_key]
    zs = values[z_key]
    
    x_log = x_key in logscale
    y_log = y_key in logscale
    z_log = z_key in logscale
        
    x_lims = get_lim( xs, x_log, x_key, )
    y_lims = get_lim( ys, y_log, y_key, )
    z_lims = get_lim( zs, z_log, z_key, )

    fig = plt.figure( figsize=(8,8), facecolor='w' )
    ax = plt.gca()

    for key in pm['used_variations']:
        
        try:
            x = xs[key]
            y = ys[key]
            z = zs[key]
        except KeyError:
            continue
        
        if np.isnan( x ) or np.isnan( y ) or np.isnan( z ):
            continue
        elif not key in pm['used_variations']:
            continue
        
        if '_' in key:
            sim_name, physics = key.split( '_' )
        else:
            sim_name = key
            physics = ''

        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
            
        s = ax.scatter(
            x,
            y,
            s = 200,
            color = c,
            marker = markers[physics]
        )

        if (x_key, y_key, z_key) not in combinations_no_annotate:
            # Annotate simulation names
            annot_args = {
                'textcoords': 'offset points',
                'fontsize': 22,
                'va': 'bottom',
                'ha': 'left',
                'xytext': ( 3, 3 ),
            }
            # Custom annotation arguments
            if ( x_key, y_key ) in custom_annot_args:
                if key in custom_annot_args[(x_key,y_key)]:
                    custom_args = custom_annot_args[(x_key,y_key)][key]
                    if custom_args is not None:
                        annot_args.update( custom_args )
                    else:
                        annot_args = None
            if annot_args is not None:
                ax.annotate(
                    text = sim_name,
                    xy = ( x, y ),
                    **annot_args
                )
            # Special cases
            if ( x_key, y_key ) == ( 'delta_pdfcosphi', 'thin_disk_frac_recent' ):
                if key == 'm11e_md':
                    ax.annotate(
                        text = 'm11a, m11c, m11d\nm11e, m11i, m11q',
                        xy = ( x+0.05, y ),
                        xytext = ( 30, 5 ),
                        textcoords = 'offset points',
                        fontsize = 22,
                        va = 'center',
                        arrowprops = {
                            'arrowstyle': '-[',
                            'lw': 1.5,
                        },
                    )
            elif ( x_key, y_key ) == ( 'thin_disk_frac_recent', 'delta_disk_frac' ):
                if key == 'm11e_md':
                    ax.annotate(
                        text = 'm11e, m11i, m11q',
                        xy = ( x+0.01, y+0.01 ),
                        xytext = ( -10, 35 ),
                        textcoords = 'offset points',
                        fontsize = 22,
                        ha = 'left',
                        va = 'center',
                        arrowprops = {
                            'arrowstyle': '-[',
                            'lw': 1.5,
                        },
                    )
                elif key == 'm11d_md':
                    ax.annotate(
                        text = 'm11a, m11c, m11d',
                        xy = ( x+0.03, y ),
                        xytext = ( 20, 0 ),
                        textcoords = 'offset points',
                        fontsize = 22,
                        va = 'center',
                        arrowprops = {
                            'arrowstyle': '-[',
                            'lw': 1.5,
                        },
                    )
        
    if y_key in [ 'delta_smooth_disk_frac', 'delta_smooth_jdisk_frac', ]:
        ax.annotate(
            text = delta_f_aligned_label( y_key ),
            xy = ( 0, 1 ),
            xycoords = 'axes fraction',
            xytext = ( 5, -5 ),
            textcoords = 'offset points',
            va = 'top',
            ha = 'left',
            fontsize = 18,
        )

        # Zero line
        ax.axhline(
            0,
            color = pm['background_linecolor'],
            linewidth = 1,
            zorder = -100,
        )

        if pm['include_proportional_line']:
            # Pro Line
            thin_disk_fracs_for_thin_disks = []
            delta_disk_frac_for_thin_disks = []
            for key in pm['used_variations']:
                item = values[x_key][key]
                if item > 0.5:
                    thin_disk_fracs_for_thin_disks.append( item )
                    delta_disk_frac_for_thin_disks.append( values[y_key][key] )
            average_thin_disk_frac_for_thin_disks = np.mean( thin_disk_fracs_for_thin_disks )
            average_delta_disk_frac_for_thin_disks = np.mean( delta_disk_frac_for_thin_disks )
            ax.plot(
                [ 0, 1 ],
                [ 0, average_delta_disk_frac_for_thin_disks / average_thin_disk_frac_for_thin_disks ],
                color = pm['background_linecolor'],
                linewidth = 1,
                zorder = -100,
            )
    
    ax.set_xlabel( labels[x_key], fontsize=22 )
    ax.set_ylabel( labels[y_key], fontsize=22 )
    
    if x_log:
        ax.set_xscale( 'log' )
    if y_log:
        ax.set_yscale( 'log' )
    
    ax.set_xlim( x_lims )
    ax.set_ylim( y_lims )
    
    if ( x_key in fractions and y_key in fractions ) or ( x_key[:15] == 'delta_disk_frac' and y_key[:15] == 'delta_disk_frac' ):
        ax.set_aspect( 'equal' )
        ax.plot(
            [ 0, 1 ],
            [ 0, 1 ],
            color = '.2',
            linewidth = 1,
        )
    
    # Legend
#     legend_elements = [
#         Line2D([0], [0], marker=markers[_], color='w', label=marker_labels[_], markerfacecolor='k', markersize=15)
#         for _ in markers.keys()
#     ]
#     ax.legend(
#         handles=legend_elements,
#         prop = {'size': 16 },
#     )
    
    # Colorbar
    if not z_log:
        norm_class = mpl.colors.Normalize
    else:
        norm_class = mpl.colors.LogNorm
    norm = norm_class(vmin=z_lims[0], vmax=z_lims[1])
    divider = make_axes_locatable( ax )
    cax = divider.append_axes( "right", pad=0.05, size='5%' )
    cbar = mpl.colorbar.ColorbarBase( cax, cmap=cmap, norm=norm, )
    
    # Colorbar label
    cax.annotate(
        text = labels[z_key],
        xy = (1,1),
        xytext = ( 0, 5 ),
        xycoords = 'axes fraction',
        textcoords = 'offset points',
        fontsize = 22,
        ha = 'right',
        va = 'bottom',
    )
    
    if (x_key, y_key, z_key) in combinations_to_save:
        plotting.save_fig(
            out_dir = os.path.join( pm['figure_dir'], 'prevalence' ),
            save_file = '{}_v_{}.pdf'.format( y_key, x_key ),
            fig = fig,
        )

## Multi-Panel

In [ ]:
all_multipanel_combinations = [
    [
        ( 'mvir', 'thin_disk_frac_recent', 'delta_smooth_disk_frac', ),
        ( 'tcool_tff', 'thin_disk_frac_recent', 'delta_smooth_disk_frac', ),
    ],
    [
        ( 'mvir', 'superthin_disk_frac_recent', 'delta_smooth_disk_frac', ),
        ( 'tcool_tff', 'superthin_disk_frac_recent', 'delta_smooth_disk_frac', ),
    ],
    [
        ( 'mvir', 'superthin_disk_frac_recent', 'delta_smooth_jdisk_frac', ),
        ( 'tcool_tff', 'superthin_disk_frac_recent', 'delta_smooth_jdisk_frac', ),
    ],
    [
        ( 'spin_param', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'f_j_disk', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'delta_smooth_disk_frac', 'superthin_disk_frac_recent', 'mvir', ),
    ],
    [
        ( 'spin_param', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'f_j_disk', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'f_j_galaxy', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'delta_smooth_disk_frac', 'superthin_disk_frac_recent', 'mvir', ),
    ],
    [
        ( 'spin_param', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'f_j_disk', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'f_j_galaxy', 'superthin_disk_frac_recent', 'mvir', ),
        ( 'delta_smooth_jdisk_frac', 'superthin_disk_frac_recent', 'mvir', ),
    ],
]
multipanel_to_save = 1
multipanel_to_save2 = 4

In [ ]:
for m, multipanel_combinations in enumerate( all_multipanel_combinations ):

    n_cols = len( multipanel_combinations )

    if n_cols > 2:
        sidelength = 5
    else:
        sidelength = 7.5
    fig = plt.figure( figsize=(sidelength * ( n_cols - 1 ) + 1.15 * sidelength, sidelength/1.3), facecolor='w' )
    main_ax = plt.gca()

    gs = gridspec.GridSpec( 1, n_cols )
    gs.update( wspace=0.0001 )

    invalid = []
    for i, (x_key, y_key, z_key) in enumerate( multipanel_combinations ):
        
        if z_key in [ 'delta_smooth_disk_frac', 'delta_smooth_jdisk_frac' ]:
            cmap = palettable.matplotlib.Plasma_20.mpl_colormap
        else:
            cmap = palettable.matplotlib.Viridis_20.mpl_colormap

        panel_invalid = []

        ax = plt.subplot(gs[0,i])

        xs = values[x_key]
        ys = values[y_key]
        zs = values[z_key]

        x_log = x_key in logscale
        y_log = y_key in logscale
        z_log = z_key in logscale

        x_lims = get_lim( xs, x_log, x_key, 1.2, )
        y_lims = get_lim( ys, y_log, y_key, 1.1, 2 )
        z_lims = get_lim( zs, z_log, z_key, )

        for key in pm['used_variations']:

            try:
                x = xs[key]
                y = ys[key]
                z = zs[key]
            except KeyError:
                panel_invalid.append( key )
                continue

            if np.isnan( x ) or np.isnan( y ) or np.isnan( z ):
                continue

            elif not key in pm['used_variations']:
                continue

            if '_' in key:
                sim_name, physics = key.split( '_' )
            else:
                sim_name = key
                physics = ''

            if not z_log:
                c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
            else:
                c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
            c = cmap( c_value )

            s = ax.scatter(
                x,
                y,
                s = 200,
                color = c,
                marker = markers[physics],
                vmin = z_lims[0],
                vmax = z_lims[1],
            )
            
            if ( key == 'm12m' ) and ( z_key != 'mvir' ):
                s_label = ax.scatter(
                    x,
                    y,
                    s = 800,
                    edgecolor = pm['lighter_background_linecolor'],
                    marker = 'o',
                    color = 'none',
                    zorder = s.get_zorder() - 1,
                    linewidth = 2,
                )
                ax.annotate(
                    text = 'm12m',
                    xy = ( x, y ),
                    xycoords = 'data',
                    xytext = ( 0, -np.sqrt(800/np.pi) ),
                    textcoords = 'offset points',
                    fontsize = 14,
                    color = pm['lighter_background_linecolor'],
                    va = 'top',
                    ha = 'center',
                    zorder = s.get_zorder() - 1,
                )

    #         # Annotate simulation names
    #         annot_args = {
    #             'textcoords': 'offset points',
    #             'fontsize': 22,
    #             'va': 'bottom',
    #             'ha': 'left',
    #             'xytext': ( 3, 3 ),
    #         }
    #         # Custom annotation arguments
    #         if ( x_key, y_key ) in custom_annot_args:
    #             if key in custom_annot_args[(x_key,y_key)]:
    #                 custom_args = custom_annot_args[(x_key,y_key)][key]
    #                 if custom_args is not None:
    #                     annot_args.update( custom_args )
    #                 else:
    #                     annot_args = None
    #         if annot_args is not None:
    #             ax.annotate(
    #                 s = sim_name,
    #                 xy = ( x, y ),
    #                 **annot_args
    #             )
    #         # Special cases
    #         if ( x_key, y_key ) == ( 'delta_pdfcosphi', 'thin_disk_frac_recent' ):
    #             if key == 'm11e_md':
    #                 ax.annotate(
    #                     s = 'm11a, m11c, m11d\nm11e, m11i, m11q',
    #                     xy = ( x+0.05, y ),
    #                     xytext = ( 30, 5 ),
    #                     textcoords = 'offset points',
    #                     fontsize = 22,
    #                     va = 'center',
    #                     arrowprops = {
    #                         'arrowstyle': '-[',
    #                         'lw': 1.5,
    #                     },
    #                 )

        ax.set_xlabel( labels[x_key], fontsize=22 )
        if ax.is_first_col():
            ax.set_ylabel( labels[y_key], fontsize=22 )

        # Zero line
        if y_key == 'delta_disk_frac':
            ax.axhline(
                0,
                color = pm['background_linecolor'],
                linewidth = 1,
                zorder = -100,
            )

        if x_log:
            ax.set_xscale( 'log' )
        if y_log:
            ax.set_yscale( 'log' )

        ax.set_xlim( x_lims )
        ax.set_ylim( y_lims )

        if not ax.is_first_col():
            ax.tick_params( left=False, labelleft=False )

#         if ax.is_first_col():
#             # Legend
#             legend_elements = [
#                 Line2D([0], [0], marker=markers[_], color='w', label=marker_labels[_], markerfacecolor='k', markersize=15)
#                 for _ in markers.keys()
#             ]
#             ax.legend(
#                 handles=legend_elements,
#                 prop = {'size': 16 },
#             )

        if ax.is_last_col():
            # Colorbar
            if not z_log:
                norm_class = mpl.colors.Normalize
            else:
                norm_class = mpl.colors.LogNorm
            norm = norm_class(vmin=z_lims[0], vmax=z_lims[1])
            divider = make_axes_locatable( ax )
            cax = divider.append_axes( "right", pad=0.05, size='5%' )
            cbar = mpl.colorbar.ColorbarBase( cax, cmap=cmap, norm=norm, )

            # Colorbar label
            cax.annotate(
                text = 'color: '+ labels[z_key],
                xy = (1,1),
                xytext = ( 0, 5 ),
                xycoords = 'axes fraction',
                textcoords = 'offset points',
                fontsize = 22,
                ha = 'right',
                va = 'bottom',
            )
            
        if x_key == 'mstar':
            ax.set_xticks( [ 1e8, 1e9, 1e10, 1e11 ])
            locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=12)
            ax.xaxis.set_minor_locator(locmin)
            ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

        # Behavior for fractions/not fractions plot
        if x_key in fractions and y_key in fractions:
            ax.set_aspect( 'equal' )
            ax.plot(
                [ 0, 1 ],
                [ 0, 1 ],
                color = '.2',
                linewidth = 1,
                zorder = -100,
            )
        else:
            tick_values = ax.get_xticks()
            new_ticklabels = []
            modified = False
            for i, tick_label in enumerate( ax.get_xticklabels() ):
                replace_values = [ 0.01, 0.1, 1., 10., 100. ]
                replace_strs = [ '0.01', '0.1', '1', '10', '100' ]
                for j, replace_val in enumerate( replace_values ):
                    if np.isclose( tick_values[i], replace_val ):
                        tick_label.set_text( replace_strs[j] )
                        modified = True
                new_ticklabels.append( tick_label )
            if modified:
                ax.set_xticklabels( new_ticklabels )

        invalid.append( panel_invalid )

    if m == multipanel_to_save:
        plotting.save_fig(
            out_dir = os.path.join( pm['figure_dir'], 'prevalence' ),
            save_file = 'aligned_fraction_vs_galaxy_props.pdf',
            fig = fig,
        )
    if m == multipanel_to_save2:
        plotting.save_fig(
            out_dir = os.path.join( pm['figure_dir'], 'prevalence' ),
            save_file = 'thin_disk_vs_angular_momentum_props.pdf',
            fig = fig,
        )

## Connected Before-After

In [ ]:
multi_values = {
    'sigma_cosphi': sigma_cosphi,
    'med_cosphi': med_cosphi,
    'abs_med_cosphi': abs_med_cosphi,
    'pdfcosphi_0': data['cosphi']['pdf(cos theta=0)'],
    'disk_frac': disk_frac,
}

In [ ]:
rd_bu_cmap = palettable.scientific.diverging.Berlin_3_r.mpl_colors

In [ ]:
annotate = False

In [ ]:
for x_key in multi_values.keys():

    fig = plt.figure( figsize=(8,8), facecolor='w' )
    ax = plt.gca()

    y_key = 'superthin_disk_frac_recent'

    xs = multi_values[x_key]
    xs_pre = xs.inner_item( 0 )
    xs_post = xs.inner_item( -1 )
    ys = values[y_key]

    x_log = x_key in logscale
    y_log = y_key in logscale
    z_log = z_key in logscale

    x_lims = get_lim( xs, x_log, x_key, 1.1 )
    y_lims = get_lim( ys, y_log, y_key, 1.1 )

    for key in pm['used_variations']:

        try:
            x_pre = xs_pre[key]
            y = ys[key]
        except KeyError:
            continue

        if np.isnan( x_pre ):
            continue

        if '_' in key:
            sim_name, physics = key.split( '_' )
        else:
            sim_name = key
            physics = ''

        # Pre
        s = ax.scatter(
            x_pre,
            y,
            s = 100,
            color = rd_bu_cmap[0],
            marker = markers[physics]
        )
        s = ax.scatter(
            xs_post[key],
            y,
            s = 100,
            color = rd_bu_cmap[-1],
            marker = markers[physics]
        )

        if annotate:
            ax.annotate(
                text = sim_name,
                xy = ( x_pre, y ),
                xytext = ( 5, 5 ),
                textcoords = 'offset points',
                fontsize = 22,
                ha = 'left',
                va = 'bottom',
            )

        ax.arrow(
            x_pre, y,
            xs_post[key] - x_pre, 0,
    #         linewidth = 1.5,
            color = 'k',
            zorder = -100,
            width = 0.002,
            head_length = 0.06,
            head_width = 0.02,
            length_includes_head = True,
        )

    if x_key == 'sigma_cosphi':
        # Reference lines
        sigma_cosphi_sphere = 0.84 * 2 - 0.16 * 2
        ax.axvline(
            sigma_cosphi_sphere,
            color = '.5',
            linestyle = '-',
            linewidth = 1,
        )
        ax.annotate(
            text = 'spherical\ndistribution',
            xy = ( sigma_cosphi_sphere, 0.6 ),
            xycoords = 'data',
            xytext = ( -5, -5 ),
            textcoords = 'offset points',
            ha = 'right',
            va = 'top',
            fontsize = 22,
            color = '.5',
        )

    ax.set_xlabel( labels[x_key], fontsize=22 )
    ax.set_ylabel( labels[y_key], fontsize=22 )

    if x_log:
        ax.set_xscale( 'log' )
    if y_log:
        ax.set_yscale( 'log' )

    ax.set_xlim( x_lims )
    ax.set_ylim( y_lims )

    # Legend
    legend_labels = [ 'Hot', 'Cool' ]
    legend_elements = [
    Line2D([0], [0], marker='o', color='k', label=legend_labels[_], markerfacecolor=rd_bu_cmap[_], markersize=15)
    for _ in [ 0, -1 ]
    ]
    ax.legend(
    handles=legend_elements,
    prop = {'size': 16 },
    )

## Combined

In [ ]:
multi_values = {
    'smooth_disk_frac': smooth_disk_frac,
    'smooth_jdisk_frac': smooth_jdisk_frac,
}

In [ ]:
t_tchange_centers = list( data['cosphi']['t_tchange_centers'].values() )[0]

In [ ]:
rd_bu_colors = palettable.scientific.diverging.Berlin_3_r.mpl_colors
rd_bu_colors_base = palettable.scientific.diverging.Roma_20
rd_bu_colors = rd_bu_colors_base.mpl_colors
rd_bu_cmap = rd_bu_colors_base.mpl_colormap

In [ ]:
cmap = palettable.matplotlib.Viridis_20.mpl_colormap

In [ ]:
annotate = False

In [ ]:
sims_subset = []

In [ ]:
norm_instance = matplotlib.colors.Normalize( vmin=pm['aligned_fraction_pre_dt']/1000., vmax=pm['aligned_fraction_post_dt']/1000. )

In [ ]:
n_cols = 2
n_rows = 1
scale = 1.5

fig = plt.figure( figsize=(6*n_cols*scale,5*scale), facecolor='w' )
main_ax = plt.gca()

gs = gridspec.GridSpec( 1, n_cols )
#     gs.update( wspace=0.0001 )

x_key = pm['thin_disk_frac_key'].replace( 'fraction', 'frac' )
z_key = 'mvir'
y_key = 'smooth_disk_frac'

xs = values[x_key]
zs = values[z_key]
ys = values[y_key]
y_multis = multi_values[y_key]
ys_pre = verdict.Dict({})
ys_post = verdict.Dict({})
for key, y in ys.items():
    interp = scipy.interpolate.interp1d( t_tchange_centers, y )
    ys_post[key] = interp( pm['aligned_fraction_post_dt'] / 1e3 )
    ys_pre[key] = interp( pm['aligned_fraction_pre_dt'] / 1e3 )
ys_delta = ys_post - ys_pre

x_log = x_key in logscale
y_log = y_key in logscale
z_log = z_key in logscale

x_lims = get_lim( xs, x_log, x_key, 1.1 )
y_lims = get_lim( ys, y_log, y_key, 1.1 )

####################################################
# Left panel
####################################################

ax = plt.subplot( gs[0,0] )

for key in pm['used_variations']:
    
    if key not in sims_subset and len( sims_subset ) > 0:
        continue

    try:
        y_pre = ys_pre[key]
        x = xs[key]
    except KeyError:
        continue

    if np.isnan( y_pre ):
        continue

    if '_' in key:
        sim_name, physics = key.split( '_' )
    else:
        sim_name = key
        physics = ''

    # Pre
    s = ax.scatter(
        x,
        y_pre,
        s = 300,
        color = rd_bu_colors[0],
        marker = markers[physics]
    )
    # Post
    s = ax.scatter(
        x,
        ys_post[key],
        s = 300,
        color = rd_bu_colors[-1],
        marker = markers[physics]
    )
    # Change along the way
    y_multi_vals = y_multis[key]
    for j, val in enumerate( y_multi_vals ):
        
        if ( t_tchange_centers[j] < pm['aligned_fraction_pre_dt'] / 1000. ) or ( t_tchange_centers[j] > pm['aligned_fraction_post_dt'] / 1000. ):
            continue
        
        valtime = t_tchange_centers[j] + data['tchange_offset'][key] / 1000.
        valcolor = rd_bu_cmap( norm_instance( valtime ) )
        
#         # Ticks
#         tick_length = 0.015
#         linewidth = 5
#         if np.isclose( t_tchange_centers[j], 0.0 ):
#             linewidth = 7
#             tick_length *= 2
#             ax.plot(
#                 [ x - tick_length / 2, x + tick_length / 2],
#                 [ val, ] * 2,
#                 linewidth = linewidth,
#                 color = rd_bu_colors[j],
#                 zorder = -10,
#             )
            
            # Markers
#             ax.scatter(
#                 [ x, ] ,
#                 [ val, ] * 2,
#                 color = rd_bu_colors[j],
#                 s = 300,
#                 marker = markers[physics]
#             )

        # Line itself
        if j != 0:
            start_point = 0.5 * ( val + y_multi_vals[j-1] )
        else:
            start_point = val
        if j != len( y_multi_vals ) - 1:
            end_point = 0.5 * ( y_multi_vals[j+1] + val )
        else:
            end_point = val
        ax.plot(
            [ x, ] * 2,
            [ start_point, end_point ],
            linewidth = 3,
            color = valcolor,
            zorder = -20,
        )
#     s = ax.scatter(
#         [ x, ] * len( y_multi_vals ),
#         y_multi_vals,
#         s = 300,
#         color = rd_bu_colors,
#         marker = markers[physics],
#     )

    # Mark center
    j_center = np.argmin(np.abs(t_tchange_centers))
    y_prev = y_multi_vals[j_center]
    linewidth = 7
    tick_length = 0.03
    if np.isclose( data['tchange_offset'][key], 0. ):
        val = y_multi_vals[j_center]
    else:
        valinterp = scipy.interpolate.interp1d( t_tchange_centers[j_center:j_center+4], y_multi_vals[j_center:j_center+4] )
        val = valinterp( data['tchange_offset'][key] / 1000. )
    ax.plot(
        [ x - tick_length / 2, x + tick_length / 2],
        [ val, ] * 2,
        linewidth = linewidth,
        color = rd_bu_cmap( norm_instance( 0. ) ),
        zorder = -10,
    )

    if annotate:
        ax.annotate(
            text = sim_name,
            xy = ( x, y_pre ),
            xytext = ( 5, 5 ),
            textcoords = 'offset points',
            fontsize = 22,
            ha = 'left',
            va = 'bottom',
        )
    ax.plot(
        [ x, x ],
        [ y_pre, ys_post[key] ],
        linewidth = 1,
        color = 'k',
        zorder = -100,
    )

if y_key == 'sigma_cosphi':
    # Reference lines
    sigma_cosphi_sphere = 0.84 * 2 - 0.16 * 2
    ax.axvline(
        sigma_cosphi_sphere,
        color = pm['background_linecolor'],
        linestyle = '-',
        linewidth = 1,
    )
    ax.annotate(
        text = 'spherical\ndistribution',
        xy = ( sigma_cosphi_sphere, 0.6 ),
        xycoords = 'data',
        xytext = ( -5, -5 ),
        textcoords = 'offset points',
        ha = 'right',
        va = 'top',
        fontsize = 22,
        color = '.5',
    )
elif ( y_key == 'disk_frac' ) or ( y_key == 'smooth_disk_frac' ):
    ax.axhline(
        pm['disk_costheta'],
        color = pm['background_linecolor'],
        linewidth = 1,
        zorder = -100,
    )
    ax.annotate(
        text = 'isotropic distribution value',
        xy = ( 0.5, pm['disk_costheta'] ),
        xycoords = 'data',
        xytext = ( 5, -15 ),
        textcoords = 'offset points',
        ha = 'right',
        va = 'top',
        fontsize = 18,
        color = pm['background_linecolor'],
        arrowprops = { 'arrowstyle': '-', 'color': '.5', },
    )
    
#     # Typical value for thin disk stars
#     ax.axhline(
#         aligned_frac['m12i_md'],
#         color = pm['background_linecolor'],
#         linewidth = 1,
#         zorder = -100,
#     )
#     ax.annotate(
#         text = 'value for thin disk stars',
#         xy = ( 0.5, aligned_frac['m12i_md'] ),
#         xycoords = 'data',
#         xytext = ( 5, -15 ),
#         textcoords = 'offset points',
#         ha = 'right',
#         va = 'top',
#         fontsize = 18,
#         color = pm['background_linecolor'],
#         arrowprops = { 'arrowstyle': '-', 'color': '.5', },
#     )


ax.set_xlabel( labels[x_key], fontsize=22 )
ax.set_ylabel( labels[y_key], fontsize=22 )

# ax.annotate(
#     text = 'Aligned mass fraction',
#     xy = ( 0, 1 ),
#     xycoords = 'axes fraction',
#     xytext = ( 5, 5 ),
#     textcoords = 'offset points',
#     fontsize = 20,
#     ha = 'left',
#     va = 'bottom',
# )

if x_log:
    ax.set_xscale( 'log' )
if y_log:
    ax.set_yscale( 'log' )

ax.set_xlim( x_lims )
ax.set_ylim( 0, 0.63 )

# Legend
rd_bu_colors_legend = palettable.scientific.diverging.Roma_3.mpl_colors
legend_labels = [
    '{} Myrs before '.format( pm['aligned_fraction_pre_dt'] ) + helpers.get_tchange_label( pm ),
    r'At ' + helpers.get_tchange_label( pm ),
    '{} Myrs after '.format( pm['aligned_fraction_post_dt'] ) + helpers.get_tchange_label( pm ),
]
legend_elements = [
Line2D([0], [0], marker='o', color='k', label=legend_labels[_], markerfacecolor=rd_bu_colors_legend[_], markersize=15)
for _ in [ 0, 1, 2 ]
]
l = ax.legend(
    handles=legend_elements,
    prop = {'size': 16 },
    loc = 'upper left',
)
l.set_alpha( 1. )

####################################################
# Right panel
####################################################

y_key = 'delta_' + y_key

x_lims = get_lim( xs, x_log, x_key, 1.2, )
y_lims = get_lim( ys_delta, y_log, y_key, 1.1, 2 )
z_lims = get_lim( zs, z_log, z_key, )

ax = plt.subplot(gs[0,1])

for key in pm['used_variations']:

    try:
        x = xs[key]
        y = ys_delta[key]
        z = zs[key]
    except KeyError:
        continue

    if np.isnan( x ) or np.isnan( y ) or np.isnan( z ):
        continue
    elif not key in pm['used_variations']:
        continue

    if '_' in key:
        sim_name, physics = key.split( '_' )
    else:
        sim_name = key
        physics = ''

    if not z_log:
        c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
    else:
        c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
    c = cmap( c_value )

    s = ax.scatter(
        x,
        y,
        s = 300,
        color = c,
        marker = markers[physics],
        zorder = np.log10( mvir[key] ) * 10,
    )
    if key == 'm12m':
        new_size = 4 * s.get_sizes()[0]
        s_label = ax.scatter(
            x,
            y,
            s = new_size,
            edgecolor = pm['lighter_background_linecolor'],
            marker = 'o',
            color = 'none',
            zorder = s.get_zorder() - 1,
            linewidth = 2,
        )
        ax.annotate(
            text = 'm12m',
            xy = ( x, y ),
            xycoords = 'data',
            xytext = ( 0, -np.sqrt(new_size/np.pi) ),
            textcoords = 'offset points',
            fontsize = 14,
            color = pm['lighter_background_linecolor'],
            va = 'top',
            ha = 'center',
            zorder = s.get_zorder() - 1,
        )
    
# Zero line
if y_key in [ 'delta_disk_frac', 'delta_smooth_disk_frac' ]:
    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 1,
        zorder = -100,
    )

ax.set_xlabel( labels[x_key], fontsize=22 )
ax.set_ylabel( labels[y_key], fontsize=22 )

# ax.annotate(
#     text = 'Change in aligned mass fraction',
#     xy = ( 0, 1 ),
#     xycoords = 'axes fraction',
#     xytext = ( 5, 5 ),
#     textcoords = 'offset points',
#     fontsize = 20,
#     ha = 'left',
#     va = 'bottom',
# )

if x_log:
    ax.set_xscale( 'log' )
if y_log:
    ax.set_yscale( 'log' )

ax.set_xlim( x_lims )
ax.set_ylim( y_lims )

if x_key in fractions and y_key in fractions:
    ax.set_aspect( 'equal' )
    ax.plot(
        [ 0, 1 ],
        [ 0, 1 ],
        color = pm['lighter_background_linecolor'],
        linewidth = 1,
    )

# # Legend
# legend_elements = [
#     Line2D([0], [0], marker=markers[_], color='w', label=marker_labels[_], markerfacecolor='k', markersize=15)
#     for _ in markers.keys()
# ]
# ax.legend(
#     handles=legend_elements,
#     prop = {'size': 16 },
# )

if y_key in [ 'delta_smooth_disk_frac', 'delta_smooth_jdisk_frac', ]:
    ax.annotate(
        text = delta_f_aligned_label( y_key ),
        xy = ( 0, 1 ),
        xycoords = 'axes fraction',
        xytext = ( 5, -5 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'left',
        fontsize = 18,
    )

# Proportional Line
if pm['include_proportional_line']:
    thin_disk_fracs_for_thin_disks = []
    delta_disk_frac_for_thin_disks = []
    for key in pm['used_variations']:
        item = values[x_key][key]
        if item > 0.5:
            thin_disk_fracs_for_thin_disks.append( item )
            delta_disk_frac_for_thin_disks.append( values['delta_smooth_disk_frac'][key] )
    average_thin_disk_frac_for_thin_disks = np.mean( thin_disk_fracs_for_thin_disks )
    average_delta_disk_frac_for_thin_disks = np.mean( delta_disk_frac_for_thin_disks )
    ax.plot(
        [ 0, 1 ],
        [ 0, average_delta_disk_frac_for_thin_disks / average_thin_disk_frac_for_thin_disks ],
        color = pm['background_linecolor'],
        linewidth = 1,
        zorder = -100,
    )

# Colorbar
if not z_log:
    norm_class = mpl.colors.Normalize
else:
    norm_class = mpl.colors.LogNorm
norm = norm_class(vmin=z_lims[0], vmax=z_lims[1])
divider = make_axes_locatable( ax )
cax = divider.append_axes( "right", pad=0.05, size='5%' )
cbar = mpl.colorbar.ColorbarBase( cax, cmap=cmap, norm=norm, )

# Colorbar label
cax.annotate(
    text = labels[z_key],
    xy = (1,1),
    xytext = ( 0, 5 ),
    xycoords = 'axes fraction',
    textcoords = 'offset points',
    fontsize = 22,
    ha = 'right',
    va = 'bottom',
)

plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'prevalence' ),
    save_file = 'aligned_fraction.pdf',
    fig = fig,
)

In [ ]:
n_cols = 2
n_rows = 1
scale = 1.5

fig = plt.figure( figsize=(6*n_cols*scale,5*scale), facecolor='w' )
main_ax = plt.gca()

gs = gridspec.GridSpec( 1, n_cols )
#     gs.update( wspace=0.0001 )

x_key = pm['thin_disk_frac_key'].replace( 'fraction', 'frac' )
z_key = 'mvir'
y_key = 'smooth_jdisk_frac'

xs = values[x_key]
zs = values[z_key]
ys = values[y_key]
y_multis = multi_values[y_key]
ys_pre = verdict.Dict({})
ys_post = verdict.Dict({})
for key, y in ys.items():
    interp = scipy.interpolate.interp1d( t_tchange_centers, y )
    ys_post[key] = interp( pm['aligned_fraction_post_dt'] / 1e3 )
    ys_pre[key] = interp( pm['aligned_fraction_pre_dt'] / 1e3 )
ys_delta = ys_post - ys_pre

x_log = x_key in logscale
y_log = y_key in logscale
z_log = z_key in logscale

x_lims = get_lim( xs, x_log, x_key, 1.1 )
y_lims = get_lim( ys, y_log, y_key, 1.1 )

####################################################
# Left panel
####################################################

ax = plt.subplot( gs[0,0] )

for key in pm['used_variations']:
    
    if key not in sims_subset and len( sims_subset ) > 0:
        continue

    try:
        y_pre = ys_pre[key]
        x = xs[key]
    except KeyError:
        continue

    if np.isnan( y_pre ):
        continue

    if '_' in key:
        sim_name, physics = key.split( '_' )
    else:
        sim_name = key
        physics = ''

    # Pre
    s = ax.scatter(
        x,
        y_pre,
        s = 300,
        color = rd_bu_colors[0],
        marker = markers[physics]
    )
    # Post
    s = ax.scatter(
        x,
        ys_post[key],
        s = 300,
        color = rd_bu_colors[-1],
        marker = markers[physics]
    )
    # Change along the way
    y_multi_vals = y_multis[key]
    for j, val in enumerate( y_multi_vals ):
        
        if ( t_tchange_centers[j] < pm['aligned_fraction_pre_dt'] / 1000. ) or ( t_tchange_centers[j] > pm['aligned_fraction_post_dt'] / 1000. ):
            continue
        
        valtime = t_tchange_centers[j] + data['tchange_offset'][key] / 1000.
        valcolor = rd_bu_cmap( norm_instance( valtime ) )
        
#         # Ticks
#         tick_length = 0.015
#         linewidth = 5
#         if np.isclose( t_tchange_centers[j], 0.0 ):
#             linewidth = 7
#             tick_length *= 2
#             ax.plot(
#                 [ x - tick_length / 2, x + tick_length / 2],
#                 [ val, ] * 2,
#                 linewidth = linewidth,
#                 color = rd_bu_colors[j],
#                 zorder = -10,
#             )
            
            # Markers
#             ax.scatter(
#                 [ x, ] ,
#                 [ val, ] * 2,
#                 color = rd_bu_colors[j],
#                 s = 300,
#                 marker = markers[physics]
#             )

        # Line itself
        if j != 0:
            start_point = 0.5 * ( val + y_multi_vals[j-1] )
        else:
            start_point = val
        if j != len( y_multi_vals ) - 1:
            end_point = 0.5 * ( y_multi_vals[j+1] + val )
        else:
            end_point = val
        ax.plot(
            [ x, ] * 2,
            [ start_point, end_point ],
            linewidth = 3,
            color = valcolor,
            zorder = -20,
        )
#     s = ax.scatter(
#         [ x, ] * len( y_multi_vals ),
#         y_multi_vals,
#         s = 300,
#         color = rd_bu_colors,
#         marker = markers[physics],
#     )

    # Mark center
    j_center = np.argmin(np.abs(t_tchange_centers))
    y_prev = y_multi_vals[j_center]
    linewidth = 7
    tick_length = 0.03
    if np.isclose( data['tchange_offset'][key], 0. ):
        val = y_multi_vals[j_center]
    else:
        valinterp = scipy.interpolate.interp1d( t_tchange_centers[j_center:j_center+4], y_multi_vals[j_center:j_center+4] )
        val = valinterp( data['tchange_offset'][key] / 1000. )
    ax.plot(
        [ x - tick_length / 2, x + tick_length / 2],
        [ val, ] * 2,
        linewidth = linewidth,
        color = rd_bu_cmap( norm_instance( 0. ) ),
        zorder = -10,
    )

    if annotate:
        ax.annotate(
            text = sim_name,
            xy = ( x, y_pre ),
            xytext = ( 5, 5 ),
            textcoords = 'offset points',
            fontsize = 22,
            ha = 'left',
            va = 'bottom',
        )
    ax.plot(
        [ x, x ],
        [ y_pre, ys_post[key] ],
        linewidth = 1,
        color = 'k',
        zorder = -100,
    )

if y_key == 'sigma_cosphi':
    # Reference lines
    sigma_cosphi_sphere = 0.84 * 2 - 0.16 * 2
    ax.axvline(
        sigma_cosphi_sphere,
        color = pm['background_linecolor'],
        linestyle = '-',
        linewidth = 1,
    )
    ax.annotate(
        text = 'spherical\ndistribution',
        xy = ( sigma_cosphi_sphere, 0.6 ),
        xycoords = 'data',
        xytext = ( -5, -5 ),
        textcoords = 'offset points',
        ha = 'right',
        va = 'top',
        fontsize = 22,
        color = '.5',
    )
elif y_key == 'disk_frac':
    ax.axhline(
        pm['disk_costheta'],
        color = pm['background_linecolor'],
        linewidth = 1,
        zorder = -100,
    )
    ax.annotate(
        text = 'isotropic distribution value',
        xy = ( 0.5, pm['disk_costheta'] ),
        xycoords = 'data',
        xytext = ( 5, -15 ),
        textcoords = 'offset points',
        ha = 'right',
        va = 'top',
        fontsize = 18,
        color = pm['background_linecolor'],
        arrowprops = { 'arrowstyle': '-', 'color': '.5', },
    )
    
#     # Typical value for thin disk stars
#     ax.axhline(
#         aligned_frac['m12i_md'],
#         color = pm['background_linecolor'],
#         linewidth = 1,
#         zorder = -100,
#     )
#     ax.annotate(
#         text = 'value for thin disk stars',
#         xy = ( 0.5, aligned_frac['m12i_md'] ),
#         xycoords = 'data',
#         xytext = ( 5, -15 ),
#         textcoords = 'offset points',
#         ha = 'right',
#         va = 'top',
#         fontsize = 18,
#         color = pm['background_linecolor'],
#         arrowprops = { 'arrowstyle': '-', 'color': '.5', },
#     )


ax.set_xlabel( labels[x_key], fontsize=22 )
ax.set_ylabel( labels[y_key], fontsize=22 )

# ax.annotate(
#     text = 'Aligned mass fraction',
#     xy = ( 0, 1 ),
#     xycoords = 'axes fraction',
#     xytext = ( 5, 5 ),
#     textcoords = 'offset points',
#     fontsize = 20,
#     ha = 'left',
#     va = 'bottom',
# )

if x_log:
    ax.set_xscale( 'log' )
if y_log:
    ax.set_yscale( 'log' )

ax.set_xlim( x_lims )
ax.set_ylim( y_lims )

# Legend
rd_bu_colors_legend = palettable.scientific.diverging.Roma_3.mpl_colors
legend_labels = [
    '{} Myrs before '.format( pm['aligned_fraction_pre_dt'] ) + helpers.get_tchange_label( pm ),
    r'At ' + helpers.get_tchange_label( pm ),
    '{} Myrs after '.format( pm['aligned_fraction_post_dt'] ) + helpers.get_tchange_label( pm ),
]
legend_elements = [
Line2D([0], [0], marker='o', color='k', label=legend_labels[_], markerfacecolor=rd_bu_colors_legend[_], markersize=15)
for _ in [ 0, 1, 2 ]
]
ax.legend(
handles=legend_elements,
prop = {'size': 16 },
)

####################################################
# Right panel
####################################################

y_key = 'delta_' + y_key

x_lims = get_lim( xs, x_log, x_key, 1.2, )
y_lims = get_lim( ys_delta, y_log, y_key, 1.1, 2 )
z_lims = get_lim( zs, z_log, z_key, )

ax = plt.subplot(gs[0,1])

for key in pm['used_variations']:

    try:
        x = xs[key]
        y = ys_delta[key]
        z = zs[key]
    except KeyError:
        continue

    if np.isnan( x ) or np.isnan( y ) or np.isnan( z ):
        continue
    elif not key in pm['used_variations']:
        continue

    if '_' in key:
        sim_name, physics = key.split( '_' )
    else:
        sim_name = key
        physics = ''

    if not z_log:
        c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
    else:
        c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
    c = cmap( c_value )

    s = ax.scatter(
        x,
        y,
        s = 300,
        color = c,
        marker = markers[physics],
        zorder = np.log10( mvir[key] ) * 10,
    )
#     if key == 'm12m':
#         new_size = 4 * s.get_sizes()[0]
#         s_label = ax.scatter(
#             x,
#             y,
#             s = new_size,
#             edgecolor = pm['lighter_background_linecolor'],
#             marker = 'o',
#             color = 'none',
#             zorder = s.get_zorder() - 1,
#             linewidth = 2,
#         )
#         ax.annotate(
#             text = 'm12m',
#             xy = ( x, y ),
#             xycoords = 'data',
#             xytext = ( 0, -np.sqrt(new_size/np.pi) ),
#             textcoords = 'offset points',
#             fontsize = 14,
#             color = pm['lighter_background_linecolor'],
#             va = 'top',
#             ha = 'center',
#             zorder = s.get_zorder() - 1,
#         )
    
# Zero line
if y_key == 'delta_disk_frac':
    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 1,
        zorder = -100,
    )

ax.set_xlabel( labels[x_key], fontsize=22 )
ax.set_ylabel( labels[y_key], fontsize=22 )

# ax.annotate(
#     text = 'Change in aligned mass fraction',
#     xy = ( 0, 1 ),
#     xycoords = 'axes fraction',
#     xytext = ( 5, 5 ),
#     textcoords = 'offset points',
#     fontsize = 20,
#     ha = 'left',
#     va = 'bottom',
# )

if x_log:
    ax.set_xscale( 'log' )
if y_log:
    ax.set_yscale( 'log' )

ax.set_xlim( x_lims )
ax.set_ylim( y_lims )

if x_key in fractions and y_key in fractions:
    ax.set_aspect( 'equal' )
    ax.plot(
        [ 0, 1 ],
        [ 0, 1 ],
        color = pm['lighter_background_linecolor'],
        linewidth = 1,
    )

# # Legend
# legend_elements = [
#     Line2D([0], [0], marker=markers[_], color='w', label=marker_labels[_], markerfacecolor='k', markersize=15)
#     for _ in markers.keys()
# ]
# ax.legend(
#     handles=legend_elements,
#     prop = {'size': 16 },
# )

if y_key in [ 'delta_smooth_disk_frac', 'delta_smooth_jdisk_frac', ]:
    ax.annotate(
        text = delta_f_aligned_label( y_key ),
        xy = ( 0, 1 ),
        xycoords = 'axes fraction',
        xytext = ( 5, -5 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'left',
        fontsize = 18,
    )

# Proportional Line
if pm['include_proportional_line']:
    thin_disk_fracs_for_thin_disks = []
    delta_disk_frac_for_thin_disks = []
    for key in pm['used_variations']:
        item = values[x_key][key]
        if item > 0.5:
            thin_disk_fracs_for_thin_disks.append( item )
            delta_disk_frac_for_thin_disks.append( values['delta_smooth_jdisk_frac'][key] )
    average_thin_disk_frac_for_thin_disks = np.mean( thin_disk_fracs_for_thin_disks )
    average_delta_disk_frac_for_thin_disks = np.mean( delta_disk_frac_for_thin_disks )
    ax.plot(
        [ 0, 1 ],
        [ 0, average_delta_disk_frac_for_thin_disks / average_thin_disk_frac_for_thin_disks ],
        color = pm['background_linecolor'],
        linewidth = 1,
        zorder = -100,
    )

# Colorbar
if not z_log:
    norm_class = mpl.colors.Normalize
else:
    norm_class = mpl.colors.LogNorm
norm = norm_class(vmin=z_lims[0], vmax=z_lims[1])
divider = make_axes_locatable( ax )
cax = divider.append_axes( "right", pad=0.05, size='5%' )
cbar = mpl.colorbar.ColorbarBase( cax, cmap=cmap, norm=norm, )

# Colorbar label
cax.annotate(
    text = labels[z_key],
    xy = (1,1),
    xytext = ( 0, 5 ),
    xycoords = 'axes fraction',
    textcoords = 'offset points',
    fontsize = 22,
    ha = 'right',
    va = 'bottom',
)

plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'prevalence' ),
    save_file = 'aligned_fraction_ang_momentum.pdf',
    fig = fig,
)

## Compare Disk Fractions

In [ ]:
cmap = palettable.matplotlib.Magma_20.mpl_colormap

In [ ]:
combinations = [
        ( 'mvir', 'thin_disk_frac_recent_diff', 'thin_disk_frac_recent' ),
        ( 'mvir', 'superthin_disk_frac_recent_diff', 'thin_disk_frac_recent' ),
#     ( 'thin_disk_frac_recent', 'superthin_disk_frac_recent', 'mvir' ),
]
combinations_to_save = combinations
combinations_no_annotate = combinations

In [ ]:
facecolors = [
    'none',
    'standard',
]
legend_labels = [
    r'$j_z/j_c(E) > 0.8$',
    r'$j_z/j_c(E) > 0.9$',
]

In [ ]:
fig = plt.figure( figsize=(8,8), facecolor='w' )
ax = plt.gca()

for i, (x_key, y_key, z_key) in enumerate( tqdm.tqdm( combinations ) ):
    
    xs = values[x_key]
    ys = values[y_key]
    zs = values[z_key]
    
    x_log = x_key in logscale
    y_log = y_key in logscale
    z_log = z_key in logscale
        
    x_lims = get_lim( xs, x_log, x_key, )
    y_lims = get_lim( ys, y_log, y_key, )
    z_lims = get_lim( zs, z_log, z_key, )

    for j, key in enumerate( pm['used_variations'] ):
        
        try:
            x = xs[key]
            y = ys[key]
            z = zs[key]
        except KeyError:
            continue
        
        if np.isnan( x ) or np.isnan( y ) or np.isnan( z ):
            continue
        elif not key in pm['used_variations']:
            continue
        
        if '_' in key:
            sim_name, physics = key.split( '_' )
        else:
            sim_name = key
            physics = ''

        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
            
        if facecolors[i] == 'standard':
            facecolor = c
        else:
            facecolor = facecolors[i]
            
        if j == 0:
            legend_label = legend_labels[i]
        else:
            legend_label = None
            
        s = ax.scatter(
            x,
            y,
            s = 200,
            facecolor = facecolor,
            edgecolor = c,
            marker = markers[physics],
            linewidth = 3,
            label = legend_label,
        )

        if (x_key, y_key, z_key) not in combinations_no_annotate:
            # Annotate simulation names
            annot_args = {
                'textcoords': 'offset points',
                'fontsize': 22,
                'va': 'bottom',
                'ha': 'left',
                'xytext': ( 3, 3 ),
            }
            # Custom annotation arguments
            if ( x_key, y_key ) in custom_annot_args:
                if key in custom_annot_args[(x_key,y_key)]:
                    custom_args = custom_annot_args[(x_key,y_key)][key]
                    if custom_args is not None:
                        annot_args.update( custom_args )
                    else:
                        annot_args = None
            if annot_args is not None:
                ax.annotate(
                    text = sim_name,
                    xy = ( x, y ),
                    **annot_args
                )
            # Special cases
            if ( x_key, y_key ) == ( 'delta_pdfcosphi', 'thin_disk_frac_recent' ):
                if key == 'm11e_md':
                    ax.annotate(
                        text = 'm11a, m11c, m11d\nm11e, m11i, m11q',
                        xy = ( x+0.05, y ),
                        xytext = ( 30, 5 ),
                        textcoords = 'offset points',
                        fontsize = 22,
                        va = 'center',
                        arrowprops = {
                            'arrowstyle': '-[',
                            'lw': 1.5,
                        },
                    )
            elif ( x_key, y_key ) == ( 'thin_disk_frac_recent', 'delta_disk_frac' ):
                if key == 'm11e_md':
                    ax.annotate(
                        text = 'm11e, m11i, m11q',
                        xy = ( x+0.01, y+0.01 ),
                        xytext = ( -10, 35 ),
                        textcoords = 'offset points',
                        fontsize = 22,
                        ha = 'left',
                        va = 'center',
                        arrowprops = {
                            'arrowstyle': '-[',
                            'lw': 1.5,
                        },
                    )
                elif key == 'm11d_md':
                    ax.annotate(
                        text = 'm11a, m11c, m11d',
                        xy = ( x+0.03, y ),
                        xytext = ( 20, 0 ),
                        textcoords = 'offset points',
                        fontsize = 22,
                        va = 'center',
                        arrowprops = {
                            'arrowstyle': '-[',
                            'lw': 1.5,
                        },
                    )
        
    if y_key in [ 'delta_smooth_disk_frac', 'delta_smooth_jdisk_frac', ]:
        ax.annotate(
            text = delta_f_aligned_label( y_key ),
            xy = ( 0, 1 ),
            xycoords = 'axes fraction',
            xytext = ( 5, -5 ),
            textcoords = 'offset points',
            va = 'top',
            ha = 'left',
            fontsize = 18,
        )

        # Zero line
        ax.axhline(
            0,
            color = pm['background_linecolor'],
            linewidth = 1,
            zorder = -100,
        )

        if pm['include_proportional_line']:
            # Pro Line
            thin_disk_fracs_for_thin_disks = []
            delta_disk_frac_for_thin_disks = []
            for key in pm['used_variations']:
                item = values[x_key][key]
                if item > 0.5:
                    thin_disk_fracs_for_thin_disks.append( item )
                    delta_disk_frac_for_thin_disks.append( values[y_key][key] )
            average_thin_disk_frac_for_thin_disks = np.mean( thin_disk_fracs_for_thin_disks )
            average_delta_disk_frac_for_thin_disks = np.mean( delta_disk_frac_for_thin_disks )
            ax.plot(
                [ 0, 1 ],
                [ 0, average_delta_disk_frac_for_thin_disks / average_thin_disk_frac_for_thin_disks ],
                color = pm['background_linecolor'],
                linewidth = 1,
                zorder = -100,
            )
            
    # Relative line
    if y_key == 'thin_disk_frac_recent_ratio':
        ax.axhline(
            1,
            color = pm['background_linecolor'],
            linewidth = 1,
            zorder = -100,
        )
    elif y_key == 'thin_disk_frac_recent_diff':
        ax.axhline(
            0,
            color = pm['background_linecolor'],
            linewidth = 1,
            zorder = -100,
        )
    
    ax.set_xlabel( labels[x_key], fontsize=22 )
    ax.set_ylabel( labels[y_key], fontsize=22 )

if x_log:
    ax.set_xscale( 'log' )
if y_log:
    ax.set_yscale( 'log' )

ax.set_xlim( x_lims )
# ax.set_ylim( y_lims )

ax.legend(
    prop = { 'size': 16, },
)

if ( x_key in fractions and y_key in fractions ) or ( x_key[:15] == 'delta_disk_frac' and y_key[:15] == 'delta_disk_frac' ):
    ax.set_aspect( 'equal' )
    ax.plot(
        [ 0, 1 ],
        [ 0, 1 ],
        color = '.2',
        linewidth = 1,
    )

# Colorbar
if not z_log:
    norm_class = mpl.colors.Normalize
else:
    norm_class = mpl.colors.LogNorm
norm = norm_class(vmin=z_lims[0], vmax=z_lims[1])
divider = make_axes_locatable( ax )
cax = divider.append_axes( "right", pad=0.05, size='5%' )
cbar = mpl.colorbar.ColorbarBase( cax, cmap=cmap, norm=norm, )

# Colorbar label
cax.annotate(
    text = labels[z_key],
    xy = (1,1),
    xytext = ( 0, 5 ),
    xycoords = 'axes fraction',
    textcoords = 'offset points',
    fontsize = 22,
    ha = 'right',
    va = 'bottom',
)
    
if (x_key, y_key, z_key) in combinations_to_save:
    plotting.save_fig(
        out_dir = os.path.join( pm['figure_dir'], 'prevalence' ),
        save_file = 'thin_disk_fracs.pdf'.format( y_key, x_key ),
        fig = fig,
    )
    

# Display Parameters

In [ ]:
sort_inds = np.argsort( default_data[pm['thin_disk_frac_key']].array() )
sorted_keys = default_data[pm['thin_disk_frac_key']].keys_array()[sort_inds[::-1]]

In [ ]:
def get_table_str( key ):
    
    if '_' in key:
        string, physics = key.split( '_' )
    else:
        string, physics = key, ''
    
    string += '  '
    printed_strs = [
        '{:.2g}'.format( default_data[pm['thin_disk_frac_key']][key] ),
        '$' + py2tex.to_tex_scientific_notation( mvir[key], 2 ) + '$',
        '$' + py2tex.to_tex_scientific_notation( mstar[key], 2 ) + '$',
        '{:.3g}'.format( rvir[key], 2 ),
        '{:.2g}'.format( delta_disk_frac[key] ),
        {
            'md': 'hydro+',
            'cr': 'CR+',
            '': 'no MD',
        }[physics],
        '?'
    ]
    
    for printed in printed_strs:
        string += '&  {}  '.format( printed )
        
    string += '  \\\\'
    
    return string

In [ ]:
for key in sorted_keys:
    
    if key not in pm['used_variations']:
        continue
        
    # Skip variants for later
    if key == 'm12i_cr' or key == 'm12i':
        continue
    
    string = get_table_str( key )
        
    print( string )

In [ ]:
for key in [ 'm12i' ]:
    
    string = get_table_str( key )
        
    print( string )